In [1]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb


In [ ]:
import math
rmse = lambda x, y: math.sqrt(mean_squared_error(x, y))

In [ ]:
train = pd.read_csv("drive/My Drive/Train.csv")
train.set_index('id', inplace=True)
test = pd.read_csv("drive/My Drive/Test.csv")
test.set_index('id', inplace=True)

FileNotFoundError: ignored

In [ ]:
train.head()


,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.0,2013,13.575957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.0,2013,11.692894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,16.237871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
train_id4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.0,2015,13.399983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train['target'].head()


id
train_id0    13.575957
train_id1    11.692894
train_id2     0.000000
train_id3    16.237871
train_id4    13.399983
Name: target, dtype: float64

In [ ]:
train['is_train'] = 0
test['is_train'] = 1
train_y = train['target']

In [ ]:
train = train.drop('target',axis=1)


In [ ]:
df = pd.concat([train, test], axis=0)


In [ ]:
df

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,6.125000e+05,0.0,0.0,0.0,6.422250e+06,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.000000e+00,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,7.200000e+05,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.000000e+00,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,3.831117e+06,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,1.319958e+06,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.000000e+00,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_id10733,222,11133,C,99,0,5714,4,2,2,2,2,1,2,2.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,1.0,180000.0,0.0,0.0,...,0.0,0.0,0.0,12382000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,3.744640e+06,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
test_id10734,500,11889,M,22,0,3701,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2013,175974433.0,445308.0,0.0,370425998.0,0.0,1415253.0,9.483231e+09,64389601.0,0.0,0.0,1.776725e+09,1.0,38976764.0,0.0,0.0,...,0.0,0.0,0.0,96353136.0,6.016020e+09,1.441891e+09,0.0,0.0,740214511.0,2.182105e+09,1.956576e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.900000e+07,0.0,0.0,0.0,490000.0,490000.0,0.0,0.0,0.0,0.0,97735.0,1.236838e+10,2013,162000.0,0.0,0.0,29160.0,NaN,NaN,NaN,NaN,1
test_id10735,197,6877,M,22,0,4115,4,2,2,2,2,1,2,1.0,1.0,1.0,1

In [ ]:
df.head()


,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTCL,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,CTR_RATISS,EXE_EXERCI,TVA_CHAFF6,TVA_CHAFF7,TVA_CHAF10,TVA_CHAF12,TVA_CAF125,TVA_CHAF15,TVA_CHAF18,TVA_CHAF22,TVA_CHAF29,TVA_CHAF36,TVA_TOTDUE,TVA_MOIDEB,TVA_CRDINI,TVA_BASIMB,TVA_DEDIMB,...,TVA_CA072018,TVA_CA132018,TVA_CA192018,ADB_MNTORD,SND_MNTPRD_I,SND_MNTTVA_I,SND_MNTDRC_I,SND_MNTAVA_I,SND_MNTTAX_I,SND_MNTPAY_I,SND_MNTAIR_I,SND_MNTPRD_E,SND_MNTTVA_E,SND_MNTDRC_E,SND_MNTAVA_E,SND_MNTTAX_E,SND_MNTPAY_E,SND_MNTAIR_E,SND_MNTPRD_A,SND_MNTTVA_A,SND_MNTDRC_A,SND_MNTAVA_A,SND_MNTTAX_A,SND_MNTPAY_A,SND_MNTAIR_A,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,is_train
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_id0,49,14185,P,99,1,6101,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,0.0,0.0,0.0,52600000.0,0.0,0.0,612500.0,0.0,0.0,0.0,6422250.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58912000.0,0.0,2530630.0,0.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id1,221,6032,P,99,0,6271,8,2,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2013,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3425295.0,0.0,10924985.0,0.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id2,181,8907,P,99,0,6102,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,31926026.0,0.0,0.0,0.0,0.0,0.0,0.0,3831117.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,34442746.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id3,252,5783,M,22,0,4403,4,2,2,2,2,1,2,1.0,1.0,1.0,1.0,1.0,2.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45425.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
train_id4,234,1477,P,99,0,6312,8,2,2,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,3,1.0,1.0,1.0,0,2015,0.0,0.0,0.0,10999653.0,0.0,0.0,0.0,0.0,0.0,0.0,1319958.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21604529.0,0.0,0.0,0.0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df['CTR_CATEGO_X'].value_counts()


M    18431
C     9642
P     7707
N       11
Name: CTR_CATEGO_X, dtype: int64

In [ ]:
df.shape


(35791, 120)

In [ ]:
from sklearn.preprocessing import LabelEncoder
df['CTR_CATEGO_X'] = LabelEncoder().fit_transform(df['CTR_CATEGO_X'])

In [ ]:
df = df.apply(lambda x: x.fillna(x.median()),axis=0)


In [ ]:
train_x = df


In [ ]:
test_x = train_x[train_x['is_train'] == 1]
train_x = train_x[train_x['is_train'] == 0]
train_x = train_x.drop(['is_train',],axis=1)
test_x = test_x.drop(['is_train',],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
train_pool = Pool(train_x, train_y) 
test_pool = Pool(test_x)
cbm0 = CatBoostRegressor(rsm=0.2, depth=13, learning_rate=0.2, eval_metric='RMSE')

In [ ]:
#XGBRegressor
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=13,
    learning_rate=0.1,
    n_estimators=2000,
    objective='reg:linear',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=4,
    **boost_params)

#LGBMRegressor
gbm0 = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=65,
    learning_rate=0.2,
    n_estimators=2000)

print("Fitting CatBoostRegressor model...")
cbm0.fit(train_pool)
print("Finished fitting CatBoostRegressor model")

print("Fitting XGBRegressor model...")
xgb0.fit(train_x, train_y)
print("Finished fitting XGBRegressor model")

print("Fitting LGBMRegressor model...")
gbm0.fit(train_x, train_y, eval_metric='rmse')
print("Finished fitting LGBMRegressor model")


Fitting CatBoostRegressor model...
0:	learn: 6.7971082	total: 507ms	remaining: 8m 26s
1:	learn: 6.6132904	total: 1.04s	remaining: 8m 41s
2:	learn: 6.4681296	total: 1.45s	remaining: 8m 1s
3:	learn: 6.3390843	total: 1.9s	remaining: 7m 52s
4:	learn: 6.2403076	total: 2.41s	remaining: 7m 58s
5:	learn: 6.1666270	total: 2.84s	remaining: 7m 50s
6:	learn: 6.0856213	total: 3.36s	remaining: 7m 56s
7:	learn: 6.0229024	total: 3.92s	remaining: 8m 5s
8:	learn: 5.9585357	total: 4.48s	remaining: 8m 13s
9:	learn: 5.9316476	total: 4.95s	remaining: 8m 10s
10:	learn: 5.8659689	total: 5.49s	remaining: 8m 13s
11:	learn: 5.8354802	total: 6.04s	remaining: 8m 17s
12:	learn: 5.8105905	total: 6.59s	remaining: 8m 20s
13:	learn: 5.7598711	total: 7.07s	remaining: 8m 18s
14:	learn: 5.7216814	total: 7.6s	remaining: 8m 18s
15:	learn: 5.6876689	total: 8.06s	remaining: 8m 15s
16:	learn: 5.6618171	total: 8.62s	remaining: 8m 18s
17:	learn: 5.6404250	total: 9.24s	remaining: 8m 24s
18:	learn: 5.6134289	total: 9.71s	remaining

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Finished fitting XGBRegressor model
Fitting LGBMRegressor model...
Finished fitting LGBMRegressor model


In [ ]:

predict_y1 = gbm0.predict(test_x)
predict_y2 = xgb0.predict(test_x)
predict_y3 = cbm0.predict(test_pool)

In [ ]:
predict_y1

array([12.82188215, 16.15672983,  8.97601965, ..., 16.52447685,
        4.102191  ,  2.54428476])

In [ ]:
predict_y = (predict_y1 + predict_y2+predict_y3)/3


In [ ]:
predict_y


array([13.00941342, 14.7532281 ,  8.54520396, ..., 14.28565611,
        5.19492939,  5.55156076])

In [ ]:
from google.colab import drive
drive.mount('drive')


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
submit = pd.DataFrame(
   {
       'id':test.index,
       'target':predict_y
   }
)

submit.to_csv('submit.csv',index=False)
!cp submit.csv drive/My\ Drive/

In [ ]:
final = pd.read_csv("submit.csv")
final.head()

,id,target
0,test_id0,13.009413
1,test_id1,14.753228
2,test_id2,8.545204
3,test_id3,11.997443
4,test_id4,14.300828
